In [1]:
import dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings.azure import AzureOpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever

dotenv.load_dotenv()

gpt_35_model = 'dev-gpt-35-turbo-sample'
embedding_model = 'dev-text-embedding-ada-002-01' 
urls = [
    'https://www.110.go.kr/consult/affect.do'

    # 'https://www.law.go.kr/lsInfoP.do?lsiSeq=39593&efYd=19971231#0000',
    # 'https://www.law.go.kr/lsInfoP.do?lsiSeq=258015&efYd=20240101#0000',
    # 'https://www.law.go.kr/lsInfoP.do?lsiSeq=260889&efYd=20240301#0000',
    # 'https://www.law.go.kr/lsInfoP.do?lsiSeq=261251&efYd=20240322#0000'
]

human_question = '''갑질 유형 중에 공공분야 내부의 갑질에 대해 설명해줘'''


In [2]:
from langchain_openai import AzureChatOpenAI

chat = AzureChatOpenAI(model=gpt_35_model, temperature=0)


In [3]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings.azure import AzureOpenAIEmbeddings

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=150, chunk_overlap=50, 
)
doc_splits = text_splitter.split_documents(docs_list)

# 벡터 데이터베이스에 문서 추가
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=AzureOpenAIEmbeddings(
        model=embedding_model# Azure OpenAIEmbedding model명
        ),
    persist_directory="./chroma_db"
)


In [4]:
# (Sparse) bm25 retriever and (Dense) faiss retriever 를 초기화 합니다.
bm25_retriever = BM25Retriever.from_documents(doc_splits)
bm25_retriever.k = 6

In [5]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vectorstore.as_retriever(search_type="similarity")], weights=[0.3, 0.7]
)

In [6]:
docs = ensemble_retriever.invoke(human_question)

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 9 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}

In [7]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

In [8]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [25]:
document_chain.invoke(
    {
        "context": [],
        "messages": [
            HumanMessage(content=human_question)
        ],
    }
)

In [21]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content=human_question)
        ],
    }
)

AttributeError: 'list' object has no attribute 'page_content'

In [28]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | ensemble_retriever,
).assign(
    answer=document_chain,
)

In [29]:
from langchain_core.messages import HumanMessage

retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content=human_question)
        ],
    }
)

KeyError: "Input to ChatPromptTemplate is missing variables {'question'}.  Expected: ['context', 'question'] Received: ['messages', 'context']"